In [5]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)

def is_flagged_word(word):
    flagged_words = [
                    'online','official','call','fantasy','contact','installation', 
                    'pay', 'manufacturer', 'delivery', 'established', 'registration',
                    'seo', 'service', 'customers', 'trading', 'company', 'buy', 'sell', 
                    'escort', 'provider', 'product', 'patients', 'website', 'sites', 
                    'free', 'affordable', 'website', 'buisness', 'installed', 'install',
                    'management', 'manufacture', 'install', 'download', 'provides',
                    'downloaded', 'services', 'development', 'drug', 'top', 'reviews',
                    'review', 'treatment', 'deliver', 'hire', 'hiring', 'supply', 'supplier', 
                    'commercial', 'discount', 'account', 'websites', 'app', 'products', 
                    'guests', 'deals'
                    ]
     if word.lower() in flagged_words: 
         return True
     return False


def is_unhelpful_word(word):
    unhelpful_words = [
                      'col','plugin','rows','state','children','content','text',
                      'type','level','children','p','h','attachment','file','layout',
                      'child','image','src','width','alt','href'
                     ]
     if word.lower() in unhelpful_words: 
         return True
     return False
        
        
def readable(split_string):
    result_data = ''
    for word in split_string:
        if word.isalpha():
            if is_unhelpful_word(word):
                pass
            else:
                result_data = result_data + ' ' + word
    return(result_data)


def description_length(description):
    description_len = len(description)
    return (description_len)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()

potential_bots = np.empty((0,2))

database = query('''SELECT id, description FROM user WHERE description != "NULL"''' )

for user in database:
    flagged_word_count = 0
    MAX_DESCRIPTION_SIZE = 1500
    joined_string = user[1]
    split_string = re.split('"|/.| ', joined_string)
    desc_len = description_length(split_string)
    
    if len(split_string) > MAX_DESCRIPTION_SIZE:
        potential_bots = np.vstack([potential_bots, user])
        continue
    
    for word in split_string:
        if(is_flagged_word(word)):
            potential_bots = np.vstack([potential_bots, [user[0], readable(split_string)]])            break
    
df = pd.DataFrame(potential_bots, columns = ['id', 'description'])
df.set_index('id', inplace = True)
print(df)


                                              description
id                                                       
1        As MSc Computer Science I enjoy forward think...
35408    multimedia explanation Hallo allerseits mein ...
68640    Shop online for prom bridesmaid wedding homec...
87383    Mathe English Infomatik Formel Deutsch buko n...
87518    was established in are an importer of fine le...
...                                                   ...
239628   Soft innovative solutions can help you achiev...
239631   My name is LaraJasmine I have been serving Ra...
239634                        a Buy Weed Online Australia
239720   a Bitcoin recovery online Well i just said le...
239777   hadir sebagai salah satu website a situs judi...

[103 rows x 1 columns]
